In [1]:
import sys
sys.path.append("..")

In [2]:
from solvers.MMR.PQK_solver import *
from circuits.circuits import *
from utils.rbf_kernel_tools import *
from squlearn.encoding_circuit import *


In [3]:
from squlearn.observables import SinglePauli
import numpy as np


def PQK_observable(num_qubits, measurement = "XYZ"):
    """"
    Returns the observable for the PQK solver

    Args:

    num_qubits (int): number of qubits in the system
    measurement (str): measurement operator to be applied to the qubits (default: "XYZ")

    Returns:
    _measurement (list): list of SinglePauli objects representing the measurement operator (shape: num_qubits*len(measurement))

    """
    if isinstance(measurement, str):
                _measurement = []
                for m_str in measurement:
                    if m_str not in ("X", "Y", "Z"):
                        raise ValueError("Unknown measurement operator: {}".format(m_str))
                    for i in range(num_qubits):
                        _measurement.append(SinglePauli(num_qubits, i, op_str=m_str))
    return _measurement

In [4]:
num_qubits = 2
num_features = 1
num_samples = 10
num_layers = 3

sigma = 1
measurement = "XYZ"
_measurement = PQK_observable(num_qubits, measurement)
print(len(_measurement), num_qubits*len(measurement))

6 6


In [5]:
from utils.rbf_kernel_tools import analytical_derivative_rbf_kernel, analytical_derivative_dot_kernel_2, matrix_rbf_dxdy_slow
from utils.rbf_kernel_tools import matrix_rbf, matrix_rbf_dx_slow, matrix_rbf_dxdx_slow

We will show that the kernel derivatives calculated by squlearn are the same as the ones analytically calculated for the separable rx fmap

In [6]:
from circuits.circuits import Separable_rx

In [7]:
sigma = 1
gamma = 1/(2*sigma**2)

experiment = { "sigma": sigma}
experiment["circuit_information"] = {"encoding_circuit": Separable_rx, "num_qubits": 1, "num_layers":1}
pqk =PQK_solver(experiment["circuit_information"],
                                Executor("pennylane"), 
                                envelope={"function": matrix_rbf, 
                                            "derivative_function": matrix_rbf_dx_slow, 
                                            "second_derivative_function": matrix_rbf_dxdx_slow,
                                            "mixed_derivative_function": matrix_rbf_dxdy_slow,
                                            "sigma": experiment["sigma"]})

In [8]:
PQK_qnn = pqk.PQK_QNN()
obs_coef = []
X = [0.1, -0.41]
#X = [-0.41, 0.1]
K_f, K_dfdx, K_dfdxdx = pqk.get_PQK_kernel_derivatives(X, PQK_qnn, obs_coef, [1, 2])

[0.1, -0.41]


In [9]:
O = PQK_qnn.evaluate(X, [], [], "f")["f"]
dOdx = PQK_qnn.evaluate(X, [], [], "dfdx")["dfdx"]
dOdxdx = PQK_qnn.evaluate(X, [], [], "dfdxdx")["dfdxdx"]

In [10]:
O

array([[ 0.        , -0.09983342,  0.99500417],
       [ 0.        ,  0.39860933,  0.91712082]])

In [11]:
dOdx

array([[[ 0.        ],
        [-0.99500417],
        [-0.09983342]],

       [[ 0.        ],
        [-0.91712082],
        [ 0.39860933]]])

In [12]:
matrix_rbf(O, O, sigma)[0, 1]

0.8805086806207679

In [13]:
O.shape

(2, 3)

In [14]:
O[0]

array([ 0.        , -0.09983342,  0.99500417])

In [15]:
dOdx.shape

(2, 3, 1)

In [16]:
# Assuming dOdx has shape (N, L, 1) and the result of matrix_rbf_dx_slow has shape (N, N, L)
# Reshape dOdx to (N, L) to align the dimensions correctly for einsum
dOdx_reshaped = dOdx[:, :, 0]

# Calculate the result using einsum
K_dx_manual = np.einsum('njl, nl->nj', matrix_rbf_dx_slow(O, O, sigma), dOdx_reshaped)


In [17]:
import sympy as sp

x,y,gamma_sp = sp.symbols("x y gamma")

K = sp.exp(-gamma_sp*(2-2*sp.cos(x-y)))
Kx = sp.diff(K, x)
Kxx = sp.diff(Kx, x)

In [18]:
matrix_rbf_dxdy_slow(np.array([[0.5]]), np.array([[0.7]]), sigma=sigma)

array([[[[0.03920795]]]])

In [19]:
matrix_rbf_dxdy_slow(np.array([[0.5, 0.4]]), np.array([[0.7, 0.3]]), sigma=sigma)

array([[[[ 0.0390124, -0.0195062],
         [-0.0195062,  0.0097531]]]])

In [20]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')

In [21]:
rbf_sp = sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))
rbf_sp_1 = sp.exp(-gamma_sp*((x1-y1)**2))

In [22]:
matrix_rbf_dx_slow(np.array([[0.5, 0.4]]), np.array([[0.7, 0.3]]), sigma=sigma)

array([[[ 0.19506198, -0.09753099]]])

In [23]:
print(rbf_sp.diff(x2).diff(x1).subs({x1: 0.5, y1: 0.7, x2: 0.4, y2: 0.3, gamma_sp: gamma}))

-0.0195061982405667


In [24]:
print(rbf_sp_1.diff(x1).diff(x1).subs({x1: 0.5, y1: 0.7,  gamma_sp: gamma}))

-0.940990726374485


In [25]:
matrix_rbf_dxdy_slow(np.array([[0.5, 0.4]]), np.array([[0.7, 0.3]]), sigma=sigma)

array([[[[ 0.0390124, -0.0195062],
         [-0.0195062,  0.0097531]]]])

In [26]:
rbf_sp.diff(x).diff(y)

0

In [27]:
rbf_sp.diff(x).subs({x: 0.5, y: 0.7, gamma_sp: gamma})

0

In [28]:
x1, y1 = sp.symbols('x_1 y_1')
x2, y2 = sp.symbols('x_2 y_2')
x3, y3 = sp.symbols('x_3 y_3')
O1 = sp.symbols('O_1', cls=sp.Function)
O2 = sp.symbols('O_2', cls=sp.Function)
O3 = sp.symbols('O_3', cls=sp.Function)


In [29]:
k = sp.symbols('k', cls=sp.Function)

In [30]:
m_sin = lambda x: -sp.sin(x)
O1_ = 0
O2_ = m_sin
O3_ = sp.cos
rbf_sp = sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))
rbf_sp3 = sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2+(x3-y3)**2))

def null_func(*args):
    return 0

def rbf_sp_fun(x1, x2, y1, y2):
    return sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2))

def rbf_fun_3_variables(x1, x2, x3, y1, y2, y3):
    return sp.exp(-gamma_sp*((x1-y1)**2+(x2-y2)**2+(x3-y3)**2))

In [31]:
#k = rbf_sp_fun
#k = rbf_fun_3_variables
k = sp.symbols('k', cls=sp.Function)
pqk_fun =  k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y))
pqk_fun_dx = sp.diff(pqk_fun, x)
pqk_fun_dxdx = sp.diff(pqk_fun_dx, x)

In [32]:
#print(get_num_expr_3(everything_dx, x,y, gamma_sp).simplify())

In [33]:
#print(get_num_expr_3(everything_dxdx, x,y, gamma_sp).simplify())

In [34]:
((k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y))).diff(x, 2)).simplify().args

(Derivative(O_1(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2)),
 Derivative(O_2(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_2(x), 2)),
 Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2)),
 Derivative(O_1(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x)),
 Derivative(O_2(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x)),
 Derivative(O_3(x), (x, 2))*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_3(x)),
 2*Derivative(O_1(x), x)*Derivative(O_2(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_2(x)),
 2*Derivative(O_1(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_3(x)),
 2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x), O_3(x)))

In [35]:
first_part = ((k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y))).diff(x, 2)).simplify().args[:3]
second_part = ((k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y))).diff(x, 2)).simplify().args[3:6]
mixed_part = ((k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y))).diff(x, 2)).simplify().args[6:9]
everything_dx = (k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y)).diff(x)).simplify()
everything_dxdx = (k(O1(x), O2(x), O3(x),  O1(y), O2(y), O3(y)).diff(x, 2)).simplify()

In [36]:
rbf_sp

exp(-gamma*((x_1 - y_1)**2 + (x_2 - y_2)**2))

In [37]:
def get_num_expr(expr, x, y, gamma):
    new_expr = expr.subs(k(O2(x), O3(x), O2(y), O3(y)), rbf_sp).subs({x1:O2(x), x2:O3(x), y1:O2(y), y2:O3(y)}).doit().subs({O2:O2_, O3:O3_}).doit().subs({gamma_sp: gamma}).simplify()
    return new_expr
def get_num_expr_3(expr, x, y, gamma):
    new_expr = expr.subs(k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y)), rbf_sp3).subs({x1:O1(x), x2: O2(x), x3:O3(x), y1:O1(y), y2:O2(y), y3:O3(y)}).doit().subs({gamma_sp: gamma}).subs({O1(x):0, O1(y):0, O2(x):m_sin(x), O2(y):m_sin(y), O3(x):sp.cos(x), O3(y):sp.cos(y)}).doit()
    return new_expr
#sp_K_PQK_dxdx = everything.subs(k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y)), rbf_sp3).subs({x1:O1(x), x2:O2(x), x3:O3(x),  y1:O1(y), y2:O2(y), y3:O3(y) }).doit().subs({O1(x):0, O2:O2_, O3:O3_}).doit().subs({gamma_sp: gamma}).simplify()
sp_K_PQK_dxdx = get_num_expr_3(everything_dxdx, x, y, gamma)
first_part_num = [get_num_expr_3(first_part[0], x, y, gamma), get_num_expr_3(first_part[1], x, y, gamma), get_num_expr_3(first_part[2], x, y, gamma)]
second_part_num = [get_num_expr_3(second_part[0], x, y, gamma), get_num_expr_3(second_part[1], x, y, gamma), get_num_expr_3(second_part[2], x, y, gamma)]
mixed_part_num = [get_num_expr_3(mixed_part[0], x, y, gamma), get_num_expr_3(mixed_part[1], x, y, gamma), get_num_expr_3(mixed_part[2], x, y, gamma)]

In [38]:
first_part_num

[0,
 1.0*(1.0*(-sin(x) + sin(y))**2 - 1)*exp(-0.5*(-sin(x) + sin(y))**2 - 0.5*(cos(x) - cos(y))**2)*cos(x)**2,
 1.0*(1.0*(cos(x) - cos(y))**2 - 1)*exp(-0.5*(-sin(x) + sin(y))**2 - 0.5*(cos(x) - cos(y))**2)*sin(x)**2]

In [39]:
second_part_num

[0,
 -0.5*(-2*sin(x) + 2*sin(y))*exp(-0.5*(-sin(x) + sin(y))**2 - 0.5*(cos(x) - cos(y))**2)*sin(x),
 0.5*(2*cos(x) - 2*cos(y))*exp(-0.5*(-sin(x) + sin(y))**2 - 0.5*(cos(x) - cos(y))**2)*cos(x)]

In [40]:
mixed_part_num

[0,
 0,
 2.0*(-sin(x) + sin(y))*(cos(x) - cos(y))*exp(-0.5*(-sin(x) + sin(y))**2 - 0.5*(cos(x) - cos(y))**2)*sin(x)*cos(x)]

In [41]:
def K_separable_rx_PQK_(X,Y, gamma):
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix

def K_separable_rx_PQK_dx(X,Y, gamma):
    #-2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)

    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = -2*gamma*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))*np.sin(X[i]-Y[j])
    return gram_matrix 

def K_separable_rx_PQK_dxdx(X,Y, gamma):
    #4*gamma**2*exp(-gamma*(2 - 2*cos(x - y)))*sin(x - y)**2 - 2*gamma*exp(-gamma*(2 - 2*cos(x - y)))*cos(x - y)
    #2*gamma*(2*gamma*sin(x)**2 - 4*gamma*sin(x)*sin(y)*cos(x - y) + 2*gamma*sin(y)**2 - cos(x - y))*exp(-gamma*(2 - 2*cos(x - y)))
    gram_matrix = np.zeros((len(X), len(Y)))    
    for i in range(len(X)):
        for j in range(len(Y)):
            gram_matrix[i,j] = 2*gamma*(2*gamma*np.sin(X[i])**2 - 4*gamma*np.sin(X[i])*np.sin(Y[j])*np.cos(X[i]-Y[j]) + 2*gamma*np.sin(Y[j])**2 - np.cos(X[i]-Y[j]))*np.exp(-gamma*(2-2*np.cos(X[i]-Y[j])))
    return gram_matrix 



In [42]:
from squlearn.kernel.matrix import ProjectedQuantumKernel

In [43]:
pqk_squlearn = ProjectedQuantumKernel(Separable_rx(num_qubits=1, num_layers=1), executor=Executor("pennylane"), gamma=gamma)

In [44]:
pqk_squlearn.evaluate(X, X)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [45]:
K_separable_rx_PQK_(X,X, gamma)

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [46]:
K_f

array([[1.        , 0.88050868],
       [0.88050868, 1.        ]])

In [47]:
K_separable_rx_PQK_dx(X,X,gamma)

array([[-0.       , -0.4298443],
       [ 0.4298443, -0.       ]])

In [48]:
K_dfdx

array([[ 0.       , -0.4298443],
       [ 0.4298443,  0.       ]])

In [49]:
K_separable_rx_PQK_dxdx(X,X,gamma)

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [50]:
K_dfdxdx

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])

In [51]:
print("first part", first_part_num[0].subs({x: X[1], y: X[0]}) + first_part_num[1].subs({x: X[1], y: X[0]})+ first_part_num[2].subs({x: X[1], y: X[0]}))
print("first part", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}))


first part -0.695660270348201
first part -0.663877618895062


In [52]:
print("second part", second_part_num[0].subs({x: X[1], y: X[0]}) + second_part_num[1].subs({x: X[1], y: X[0]})+ second_part_num[2].subs({x: X[1], y: X[0]}))
print("second part", second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]})+ second_part_num[2].subs({x: X[0], y: X[1]}))


second part 0.112049565674586
second part 0.112049565674586


In [53]:
print("mixed part", mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))
print("mixed part", mixed_part_num[0].subs({x: X[1], y: X[0]}), mixed_part_num[1].subs({x: X[1], y: X[0]}), mixed_part_num[2].subs({x: X[1], y: X[0]}))

mixed part -0.00679085302453363
mixed part 0 0 0.0249917984286051


In [54]:
print(second_part_num[1].subs({x: X[1], y: X[0]}).evalf()), print(second_part_num[2].subs({x: X[1], y: X[0]}).evalf())

0.174942922842599
-0.0628933571680127


(None, None)

In [55]:
print("first part", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}))
print("second part", second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]})+ second_part_num[2].subs({x: X[0], y: X[1]}))
print("mixed part", mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))
print("total", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}) + second_part_num[0].subs({x: X[1], y: X[0]}) + second_part_num[1].subs({x: X[1], y: X[0]})+ second_part_num[2].subs({x: X[1], y: X[0]}) + mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))

first part -0.663877618895062
second part 0.112049565674586
mixed part -0.00679085302453363
total -0.558618906245010


In [56]:
print("total", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}) + second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]})+ second_part_num[2].subs({x: X[0], y: X[1]}) + mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))
print("total", first_part_num[0].subs({x: X[1], y: X[0]}) + first_part_num[1].subs({x: X[1], y: X[0]})+ first_part_num[2].subs({x: X[1], y: X[0]}) + second_part_num[0].subs({x: X[1], y: X[0]}) + second_part_num[1].subs({x: X[1], y: X[0]})+ second_part_num[2].subs({x: X[1], y: X[0]}) + mixed_part_num[0].subs({x: X[1], y: X[0]})+ mixed_part_num[1].subs({x: X[1], y: X[0]})+ mixed_part_num[2].subs({x: X[1], y: X[0]}))

total -0.558618906245010
total -0.558618906245010


In [57]:
#first part [[-1.         -0.69566027]
# [-0.66387762 -1.        ]]
#second part [[ 0.         -0.11204957]
# [-0.11204957  0.        ]]
#mixed part [[0. 0.]
# [0. 0.]]

In [58]:
sp_K_PQK_dxdx.subs({x: X[0], y: X[1]}) 

-0.558618906245010

In [59]:
first_part[0].args[1]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2))

In [60]:
first_part[2].args[1]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2))

In [61]:
first_part

(Derivative(O_1(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2)),
 Derivative(O_2(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_2(x), 2)),
 Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2)))

In [62]:
dummy_matrix = np.zeros((2,2, 3))
for l in range(3):
    for i in range(2):
        for j in range(2):
            dummy_matrix[i,j,l] = get_num_expr_3(first_part[l].args[1], x, y, gamma).subs({x: X[i], y: X[j]})
dummy_matrix #Works OK

array([[[-1.        , -1.        , -1.        ],
        [-0.88050868, -0.66175055, -0.87516768]],

       [[-0.88050868, -0.66175055, -0.87516768],
        [-1.        , -1.        , -1.        ]]])

In [63]:
first_part[2]

Derivative(O_3(x), x)**2*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2))

In [64]:
dummy_matrix = np.zeros((2,3))
for l in range(3):
    for i in range(2):
        dummy_matrix #Works OK

In [65]:
first_part[l].args[1]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_3(x), 2))

In [66]:
first_part[0].args[0] #Works OK

Derivative(O_1(x), x)**2

In [67]:
second_part[l].args[1]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_3(x))

In [68]:
first_part[0].args[1]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), (O_1(x), 2))

In [69]:
mixed_part[2]

2*Derivative(O_2(x), x)*Derivative(O_3(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_2(x), O_3(x))

In [70]:
analytical_derivative_dummy = np.zeros((2,2, 3))
analytical_derivative_dummy_2 = np.zeros((2,2, 3))
for l in range(3):
    for i in range(2):
        for j in range(2):
            analytical_derivative_dummy[i,j,l] = get_num_expr_3(second_part[l].args[1], x, y, gamma).subs({x: X[i], y: X[j]})
            analytical_derivative_dummy_2[i,j,l] = get_num_expr_3(first_part[l].args[1], x, y, gamma).subs({x: X[i], y: X[j]})

print("dKdO analytical derivative", analytical_derivative_dummy)
#print("dKdOdO analytical derivative 2", analytical_derivative_dummy_2)

dKdO analytical derivative [[[ 0.          0.          0.        ]
  [ 0.          0.43888316 -0.06857696]]

 [[ 0.         -0.43888316  0.06857696]
  [ 0.          0.          0.        ]]]


In [71]:
mixed_part[1].args[3]

Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_3(x))

In [72]:
analytical_derivative_dummy_dxdy = np.zeros((2,2, 3, 3))
vec = [O1(x), O2(x), O3(x)]
f = k(O1(x), O2(x), O3(x), O1(y), O2(y), O3(y))

for p in range(3):
    for l in range(3):
        for i in range(2):
            for j in range(2):
                analytical_derivative_dummy_dxdy[i,j,l, p] = get_num_expr_3(f.diff(vec[p]).diff(vec[l]), x, y, gamma).subs({x: X[i], y: X[j]})
                #analytical_derivative_dummy_dxdy[i,j,l] = get_num_expr_3(mixed_part[l].args[3], x, y, gamma).subs({x: X[i], y: X[j]})

print("dKdO analytical derivative dxdy", analytical_derivative_dummy_dxdy)
#print("dKdOdO analytical derivative 2", analytical_derivative_dummy_2)

dKdO analytical derivative dxdy [[[[-1.          0.          0.        ]
   [ 0.         -1.          0.        ]
   [ 0.          0.         -1.        ]]

  [[-0.88050868  0.          0.        ]
   [ 0.         -0.66175055 -0.03418169]
   [ 0.         -0.03418169 -0.87516768]]]


 [[[-0.88050868  0.          0.        ]
   [ 0.         -0.66175055 -0.03418169]
   [ 0.         -0.03418169 -0.87516768]]

  [[-1.          0.          0.        ]
   [ 0.         -1.          0.        ]
   [ 0.          0.         -1.        ]]]]


In [73]:
analytical_derivative_dummy_dxdy[:,:, 1, 2]

array([[ 0.        , -0.03418169],
       [-0.03418169,  0.        ]])

In [74]:
O_vector_dummy = np.zeros((2, 3))
dOdx_dummy = np.zeros((2, 3))
dOdxdx_dummy = np.zeros((2, 3))
vecdx = [O1(x).diff(x), O2(x).diff(x), O3(x).diff(x)]

for l in range(3):
    for i in range(2):
        O_vector_dummy[i,l] = get_num_expr_3(vec[l], x, y, gamma).subs({x: X[i]}, y=X[0])
        dOdx_dummy[i,l] = get_num_expr_3(vecdx[l], x, y, gamma).subs({x: X[i]}, y=X[0])
        dOdxdx_dummy[i,l] = get_num_expr_3(second_part[l].args[0], x, y, gamma).subs({x: X[i]}, y=X[0])

print(O_vector_dummy,"\n")
#print(dOdx_dummy,"\n")
#print(dOdxdx_dummy,"\n")

[[ 0.         -0.09983342  0.99500417]
 [ 0.          0.39860933  0.91712082]] 



In [75]:
O_vector_dummy[:]

array([[ 0.        , -0.09983342,  0.99500417],
       [ 0.        ,  0.39860933,  0.91712082]])

In [76]:
#mixed part 0.0249917984286051
#mixed part -0.00679085302453363

In [77]:
mixed_part[0]

2*Derivative(O_1(x), x)*Derivative(O_2(x), x)*Derivative(k(O_1(x), O_2(x), O_3(x), O_1(y), O_2(y), O_3(y)), O_1(x), O_2(x))

In [78]:
mixed_part[2].args[2]

Derivative(O_3(x), x)

In [79]:
get_num_expr_3(mixed_part[2].args[3], x, y, gamma).subs(x,X[0]).subs(y, X[1]), get_num_expr_3(mixed_part[2].args[2], x, y, gamma).subs(x,X[0]).subs(y, X[1]), get_num_expr_3(mixed_part[2].args[1], x, y, gamma).subs(x,X[0]).subs(y, X[1])

(-0.0341816877187692, -0.0998334166468282, -0.995004165278026)

In [80]:
2*get_num_expr_3(mixed_part[2].args[3], x, y, gamma).subs(x,X[0]).subs(y, X[1]) * get_num_expr_3(mixed_part[2].args[2], x, y, gamma).subs(x,X[0]).subs(y, X[1]) * get_num_expr_3(mixed_part[2].args[1], x, y, gamma).subs(x,X[0]).subs(y, X[1])

-0.00679085302453363

In [81]:
#print(analytical_derivative_dummy_dxdy[:,:, 1, 2]*2*O_vector_dummy[:,1]*O_vector_dummy[:,2])
dummy_test = np.zeros((2,2))
index_combinations_of_O = list(combinations(range(3), 2))
for p, l in index_combinations_of_O:
        for i in range(2):
            for j in range(2):
                #print(analytical_derivative_dummy_dxdy[i,j, p, l], O_vector_dummy[i,p], O_vector_dummy[i,l])
                dummy_test[i,j] += analytical_derivative_dummy_dxdy[i,j, l, p]*2*dOdx_dummy[j,p]*dOdx_dummy[j,l]


print(dummy_test)

[[ 0.          0.0249918 ]
 [-0.00679085  0.        ]]


In [82]:
PQK_qnn = pqk.PQK_QNN()
obs_coef = []
X = [0.1, -0.41]
#X = [-0.41, 0.1]

K_f, K_dfdx, K_dfdxdx = pqk.get_PQK_kernel_derivatives(X, PQK_qnn, obs_coef, [1, 2])
#The problem is in analytical 2 and mixed term

[0.1, -0.41]


In [83]:
print("first part", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}))
print("second part", second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]})+ second_part_num[2].subs({x: X[0], y: X[1]}))
print("mixed part", mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))
print("total", first_part_num[0].subs({x: X[0], y: X[1]}) + first_part_num[1].subs({x: X[0], y: X[1]})+ first_part_num[2].subs({x: X[0], y: X[1]}) + second_part_num[0].subs({x: X[0], y: X[1]}) + second_part_num[1].subs({x: X[0], y: X[1]})+ second_part_num[2].subs({x: X[0], y: X[1]}) + mixed_part_num[0].subs({x: X[0], y: X[1]})+ mixed_part_num[1].subs({x: X[0], y: X[1]})+ mixed_part_num[2].subs({x: X[0], y: X[1]}))

first part -0.663877618895062
second part 0.112049565674586
mixed part -0.00679085302453363
total -0.558618906245010


In [84]:
print("first part", first_part_num[0].subs({x: X[1], y: X[0]}) + first_part_num[1].subs({x: X[1], y: X[0]})+ first_part_num[2].subs({x: X[1], y: X[0]}))
print("second part", second_part_num[0].subs({x: X[1], y: X[0]}) + second_part_num[1].subs({x: X[1], y: X[0]})+ second_part_num[2].subs({x: X[1], y: X[0]}))
print("mixed part", mixed_part_num[0].subs({x: X[1], y: X[0]}) + mixed_part_num[1].subs({x: X[1], y: X[0]}) + mixed_part_num[2].subs({x: X[1], y: X[0]}))
print("total", first_part_num[0].subs({x: X[1], y: X[0]}) + first_part_num[1].subs({x: X[1], y: X[0]})+ first_part_num[2].subs({x: X[1], y: X[0]}) + second_part_num[0].subs({x: X[1], y: X[0]}) + second_part_num[1].subs({x: X[1], y: X[0]})+ second_part_num[2].subs({x: X[1], y: X[0]}) + mixed_part_num[0].subs({x: X[1], y: X[0]})+ mixed_part_num[1].subs({x: X[1], y: X[0]})+ mixed_part_num[2].subs({x: X[1], y: X[0]}))

first part -0.695660270348201
second part 0.112049565674586
mixed part 0.0249917984286051
total -0.558618906245010


In [85]:
K_dfdxdx

array([[-1.        , -0.55861891],
       [-0.55861891, -1.        ]])